# Named Entity Recognition Outline -- TF
This script was written to be an outline for anyone attempting to apply NER to a dataset using the Tensorflow framework and HuggingFace. 

It is important to understand that the model will take in the basic 3 BERT inputs: `input_ids`, `attention_mask`, and `token_type_ids` and will output a tensor `label_ids`. All tensors should be of length 512. The values of `label_ids` are the predicted entity classifications of `input_ids`. For example:

if `input_ids[5] = word_to_id("Edinburgh")`,

then `label_ids[5] = category_to_id("CITY")`

Thus, `label_ids` (before encoding) should look something like, `["O", "CITY", "O", "NAME", "O", ...]` where "O" stands for "outside of entity". But, of course `label_ids` should be a numerical tensor, encoded like this `[0, 1, 0, 2, 0, ...]` where `{0:"O", 1:"CITY", 2:"NAME"}`. Because we are not using one-hot encoding in the previous tensor, we must use `sparse categorical crossentropy` for our `loss` and `metrics` during model compilation.

One of the most important things to consider in the NER classification process is the data preprocessing stage. In order to train a BERT NER model, one must provide their own y-values or `label_ids`. The default BERT tokenizer will split up unknown words into smaller pieces, prepending "##" to the token to signify a broken-up word. When this happens, we must somehow be able to detect this and allow the entity label to span multiple indexes. For example, if "Edinburgh" were split into `[..., "Edin", "##burgh", ...]`, we would need to add two "CITY" labels instead of one, like this `[..., "CITY", "CITY", ...]`. Furthermore, the default tokenizer splits up punctuation and gives punctuation their own token ids. We must also be able to detect punction and assign "O", or "outside of entity" label ids to these tokens. I added this functionality to `entity_label_adjustment` within this notebook.

## Functions

In [ ]:
#@title Progress Counter:
#@markdown class/function to print percentage progress after each iteration in loop

# Progress Counter
import sys

class progressCounter():

  def __init__(self, num_iterations):
    self.progress = 0
    self.N = num_iterations
    self.calls = 0

  def check_pt(self):
    self.calls+=1
    curr_progress = int(self.calls/(self.N-1) * 100)
    if curr_progress - self.progress > 0:
      self.progress = curr_progress
      sys.stdout.write("\rProgress: {0}%".format(self.progress))
      sys.stdout.flush()

In [ ]:
#@title BERT tokenizer entity label adjustment function
#@markdown function to detect and expand entity labels to take into account punctuation and words split-up by tokenizer

#@markdown when function detects punctuation or BERT special tokens, function applies label 'O', or "outside of entity"

#@markdown when function detects split-up word (Ex. `[..., Yu, ##k, ##ko, ...]`), function splits label three ways (Ex. `[..., NAME, NAME, NAME, ...]`)

import re
import string
# takes in a list of input_ids and a list of entity labels and adapts entity label array to tokenizer
def entity_label_adjustment(seq_input_ids, seq_labels, tokenizer, label_dict):
  special_tokens = ['[CLS]', '[SEP]', '[PAD]']
  seq_labels = seq_labels.copy()
  seq_tokens = tokenizer.convert_ids_to_tokens(seq_input_ids)
  for i in range(len(seq_tokens)):
    if seq_tokens[i] in special_tokens or seq_tokens[i] in string.punctuation: # defnitely outside of entity
      seq_labels.insert(i, label_dict["O"])
      continue
    m = re.search("^##[a-zA-Z]", seq_tokens[i])
    if m != None:
      seq_labels.insert(i, seq_labels[i-1])
  return seq_labels

## Import NER Classifier & Tokenizer

In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertForTokenClassification
import tensorflow as tf
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForTokenClassification.from_pretrained("bert-base-cased", output_attentions = False, output_hidden_states = False, num_labels=3) # <-- number labels required to equal number of labels in dataset

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 890kB 9.4MB/s 
     |████████████████████████████████| 3.0MB 16.7MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d4cfd82274edc38b413c9d38b58f2dfcda32ff27e63ff0b662b9b09e51a0a06f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertForTokenClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data

In [ ]:
#@title Dataset Placeholder
#@markdown Example dataset inside
categories = ['soc.religion.christian']
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42)
data = data.data

In [ ]:
# Another example
# Use non-english name to force tokenizer splitting
data = ["Hello, my name is Yukko."] # Yukko becomes [Yu, ##k, ##ko] when tokenized
labels = [ "O", "O", "N", "O", "NAME" ]
label_dict = {"O" : 0, "NAME" : 1, "N" : 2 }
# translate data set to ids
label_ids = [[ 0, 0, 2, 0, 1 ]]

In [ ]:
dict_data = tokenizer(data, return_tensors="tf", truncation=True, padding='max_length', max_length=512)

In [ ]:
# apply entity label adjustment to all input/label_ids
dict_data["label_ids"] = [0] * len(dict_data["input_ids"])
for i in range(len(dict_data["input_ids"])):
  dict_data["label_ids"][i] = tf.convert_to_tensor(     entity_label_adjustment(dict_data["input_ids"][i],label_ids[i],tokenizer, label_dict)        , dtype=tf.int8 )
  if len(dict_data["label_ids"][0]) != len(dict_data["input_ids"][0]):
    print('Error in entity label adjustment!!!')
    print('Unknown character encountered: recommended update adjustment function')
    break

dict_data["label_ids"] = tf.reshape(dict_data["label_ids"], (1,512) ) # reshape for training

## Configure, Compile, and Train

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model_history = model.fit( [dict_data["input_ids"], dict_data["attention_mask"], dict_data["token_type_ids"]], [dict_data["label_ids"]], verbose=1, batch_size=1, epochs=1)

Epoch 1/3
1/1 [==============================] - 0s 2ms/step - loss: 1.0147 - accuracy: 0.7070
Epoch 2/3
1/1 [==============================] - 0s 1ms/step - loss: 0.0571 - accuracy: 0.9902
Epoch 3/3
1/1 [==============================] - 0s 1ms/step - loss: 0.0488 - accuracy: 0.9922


## Other

In [ ]:
#@title Print Useful Variables
ids = dict_data.input_ids[0][0:15]
input_ids = dict_data.input_ids[0][0:15]
words = tokenizer.convert_ids_to_tokens(ids)
output_labels = dict_data.label_ids[0][0:15]

print(words)
print(input_ids)
print(output_labels)

['[CLS]', 'Hello', ',', 'my', 'name', 'is', 'Yu', '##k', '##ko', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tf.Tensor(
[  101  8667   117  1139  1271  1110 10684  1377  2718   119   102     0
     0     0     0], shape=(15,), dtype=int32)
tf.Tensor([0 0 0 0 2 0 1 1 1 0 0 0 0 0 0], shape=(15,), dtype=int8)


In [ ]:
#@title Tests Cases
sentence = "Hello, my name is Yukko."
label_ids = [ 0, 0, 2, 0, 1 ]
label_dict = {"O" : 0, "NAME" : 1, "N" : 2 }
dict_data = tokenizer("Hello, my name is Yukko.", return_tensors="tf", truncation=True, padding='max_length', max_length=512)
ids = dict_data.input_ids[0][0:12]
words = tokenizer.convert_ids_to_tokens(ids)
print(words)
seq_labels = entity_label_adjustment(ids,label_ids,tokenizer)
print(seq_labels)
print(len(seq_labels))
print(len(ids))